In [1]:
import pandas as pd
import numpy as np
from math import log
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [6]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = np.array(items_weights['item_id'])
    weight = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, replace=False, p=weight)
    
    return recs.tolist()

In [7]:
items_weights = data_test.groupby('item_id')['sales_value'].sum().reset_index()
items_weights.columns=['item_id', 'sales_value']
items_weights['weight'] = items_weights['sales_value'] / items_weights['sales_value'].sum()
items_weights.drop(['sales_value'], axis=1, inplace=True)
items_weights.head(2)

,item_id,weight
0,29512,0.000003
1,30356,0.000006


In [8]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head()

Wall time: 3.34 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6379559, 832330, 7167965, 15452085, 5995635]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[7441478, 15797205, 7025150, 887162, 12604797]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[928550, 731106, 969703, 48593, 1826149]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1193378, 968702, 5577760, 869689, 1079510]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[918534, 17214679, 860659, 1137979, 9932509]"


In [9]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head()

Wall time: 612 ms


,user_id,actual,random_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6379559, 832330, 7167965, 15452085, 5995635]","[1067820, 15926878, 1098694, 5569374, 10284972]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[7441478, 15797205, 7025150, 887162, 12604797]","[7464084, 1017288, 868888, 909714, 1074524]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[928550, 731106, 969703, 48593, 1826149]","[1034176, 1029743, 900248, 867648, 919423]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1193378, 968702, 5577760, 869689, 1079510]","[924770, 15596772, 7152901, 13417843, 15596513]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[918534, 17214679, 860659, 1137979, 9932509]","[9836106, 6534178, 844179, 8011397, 902944]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [10]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[1120297, 12672919, 865377, 14074566, 1073212]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[12985723, 13511470, 976405, 6920289, 953154]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [18]:
def precision_at_k(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / recommended_list.size
    
    
    return precision


In [19]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0:random_recommendation
0.0:popular_recommendation
0.0:itemitem
0.0:cosine
0.0:tfidf
0.0:own_purchases
